In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import powerlaw as pwl

In [6]:
os.chdir('../../../Downloads/New_datasets-20240309T152559Z-001/New_datasets/')

In [7]:
#num: is the baby's number,
#sil: is the kind of silence,
#theta: is the threshold

#The trials of all babies in silence X are contained in the trial_silenceX folder where each column is a good channel series 

def avalanche(num, sil, theta):
    #We import the z-score normalized trials of a specific baby according to his number and the kind of silence (1 or 2)
    #The number of good channels change in each baby 
    fl1=pd.read_table("./trials_silence"+str(sil)+"/bb"+str(num)+"s"+str(sil)+".dat", sep="\s+", header = None)
    
    #We create a time array according to the lenght of fl1
    t=np.arange(0,len(fl1)/500,0.002)
    
    #We convert the table to an array
    fl1 = np.asarray(fl1)
    #fl1.shape
    #type(t), type(fl1)
    
    #We create a boolian array where all channels coincide simultaneously being grater than a threshold
    #len(fl1[0]) is the number of good channels 
    final_t = [np.array(np.abs(fl1[:,i]) > theta, dtype=float) for i in range(len(fl1[0]))]
    final_t = np.array(sum(final_t), dtype=bool)
    
    #We convert the boolian array to a float numeric
    final_t = np.array(final_t, dtype = float)
    
    #The key part
    av_indice_start = np.where((final_t[1:] - final_t[:-1]) >0)[0] # These are the indices where an avalanche begins
    av_indice_end = np.where((final_t[1:] - final_t[:-1])< 0)[0]   # hese are the indices where an avalanche ends
    
    #We modify the content of both av_indice_start and av_indice_end depending on the 
    #initial and final values of final_t
    
    #Here both av_indice_start and av_indice_end register correctly all avalanche begginings and ends
    if final_t[0]==0 and final_t[-1]==0: 
        av_indice_start = av_indice_start
        av_indice_end = av_indice_end  # No changes on both arrays since they are correct at possitions and lenghts 
    
    #Here av_indice_end does not register the position of the end of the last avalanche  
    elif final_t[0]==0 and final_t[-1]==1:
        av_indice_end = np.append(av_indice_end,len(fl1[:,0])-1) 
    
    #Here av_indice_start does not register the position of the begining of the first avalanche
    elif final_t[0]==1 and final_t[-1]==0:
        av_indice_start = np.insert(av_indice_start,0,0)
    
    #Here av_indice_start does not register the position of the begining of the first avalanche
    # and av_indice_end does not register the position of the end of the last avalanche
    
    elif final_t[0]==1 and final_t[-1]==1:
        av_indice_start = np.insert(av_indice_start,0,0)
        av_indice_end = np.append(av_indice_end,len(fl1[:,0])-1)
        
    #flag = True if len(av_indice_start) < len(av_indice_end) else False
    #if flag:
    #    av_indice_start = np.insert(av_indice_start,0,0) ## I corrected here
    
    #if not flag:
    #    av_indice_end = np.append(av_indice_end,len(fl1[:,0])-1)               
        
    global_signal = (sum([np.abs(fl1[:,i]) for i in range(len(fl1[0]))]))
    
    avalanche_sizes = []
    avalanche_dur = []

    for s in range(len(av_indice_start)):
        if len(av_indice_start) != len(av_indice_end):
            print('Error, they must be of the same length')
            break
        avalanche_sizes.append(np.sum(global_signal[av_indice_start[s]:av_indice_end[s]]))
        avalanche_dur.append(t[av_indice_end[s]]- t[av_indice_start[s]])
        
    return np.array(avalanche_sizes), np.array(avalanche_dur) 
    

In [5]:
# Here we run the function for all babies in both kind of silences
# ssil is the kind of silence
# tth is the value of the threshold 

def export_avalanche(ssil, tth):
    #We take into consideration both groups of babies at both silences 
    bbs1=[int(x) for x in open("./bbs_s1.dat", "r").read().split("\n")]
    bbs2=[int(x) for x in open("./bbs_s2.dat", "r").read().split("\n")]

    bbs1=np.array(bbs1) #The group of babies for silence 1  
    bbs2=np.array(bbs2) #The group of babies for silence 2
    
    if ssil == 1:
        group_babies = bbs1
    else:
        group_babies = bbs2
        
    print("Processing babies in silence "+str(ssil)+" with theta = "+str(tth))
    ss = []
    dd = []
    for i in group_babies:
        print("Baby %s" %i)
        #fl2=open("./ben_avalanches_silence"+str(ssil)+"/bb"+str(i)+"th"+str(tth)+".dat", "w")
        avs, tms = avalanche(i, ssil, tth)
        ss.append(avs)
        dd.append(tms)
        #for j in range(len(avs)):
        #    fl2.write("%.3f %3f\n" %(avs[j], tms[j]))
        #fl2.close()
        print("done")
    return ss,dd

In [ ]:
s,d = export_avalanche(1,2)


Processing babies in silence 1 with theta = 2
Baby 5
done
Baby 6
done
Baby 9
done
Baby 10
done
Baby 11
done
Baby 12
done
Baby 16
done
Baby 17
done
Baby 18
done
Baby 19
done
Baby 21
done
Baby 30


In [ ]:
value = False

plt.rcParams['font.size'] = 16
fig = plt.figure(figsize = (12,12))
axs = fig.add_subplot(2,2,1)
axd = fig.add_subplot(2,2,2)
axsh = fig.add_subplot(2,2,3)
axdh = fig.add_subplot(2,2,4)
alphas = []

taus = []
totals = []
totald  =[]
i = 0
os.chdir('../../../Downloads/avalanches_in_silences/avalanches_silence1/')
for file in os.listdir():
    if 'th2' in file:
        print(file)
        lista = open(file, 'r')
        datContent = [i.split() for i in lista.readlines()]
        newdata = np.asarray(datContent, dtype = float)
        s = newdata[:,0]
        d = newdata[:,1]
        data = s
        fit = pwl.Fit(data,xmin =(1,100),xmax= max(data),parameter_range= {"alpha":[1,4]},discrete = value)
        exp = fit.power_law.alpha
        pwl.plot_pdf(data,color = 'tab:red', marker = '.', ax = axs, alpha = 0.3)
        #fit.power_law.plot_pdf(data,color = 'tab:red' ,linestyle = '--', ax = axs)
        axs.set_xlabel(r'Avalanche sizes')
        axs.set_ylabel(r'pdf')
        #axs.annotate("alpha= {}".format(round(exp,2)),[0.8,0.5])
        alphas.append(exp)
        data = d
        fit = pwl.Fit(data,xmin =(0,0.05),xmax= max(data),parameter_range= {"alpha":[1,4]},discrete = value)
        exp = fit.power_law.alpha
        pwl.plot_pdf(data,color = 'tab:green', marker = '.', ax = axd, alpha = 0.3)
        #fit.power_law.plot_pdf(data,color = 'tab:green' ,linestyle = '--', ax = axd, alpha = 0.6)
        axd.set_xlabel(r'Avalanche lifetimes')
        axd.set_ylabel(r'pdf')
        #axd.annotate("alpha= {}".format(round(exp,2)),[0.8,0.5])
        taus.append(exp)
        #if i == 1: break
        axsh.hist(alphas, bins = 15, color = 'red', alpha = .5, discrete = True)
        axsh.set_xlabel('Alphas')
        axsh.set_ylabel('pdf')
        
        axdh.hist(taus, bins = 15, color = 'green', alpha = .5, discrete = True)
        axdh.set_xlabel('Taus')
        axdh.set_ylabel('pdf')
        
        
        
        totals += newdata[:,0].tolist()
        totald += newdata[:,1].tolist()
        i+=1
        
        

totals = np.asarray(totals)
totald = np.asarray(totald)

data = totals
fit = pwl.Fit(data,xmin =(1,100),xmax= max(data),parameter_range= {"alpha":[1,4]},discrete = value)
exp = fit.power_law.alpha
axs.annotate("Pooled alpha= {}".format(round(exp,2)),[0.8,0.5], xycoords = 'axes fraction',fontsize = 11)

pwl.plot_pdf(data,color = 'tab:red', marker = '.', ax = axs)
fit.power_law.plot_pdf(data,color = 'tab:red' ,linestyle = '--',ax = axs)


data = totald
fit = pwl.Fit(data,xmin =(0,0.05),xmax= max(data),parameter_range= {"alpha":[1,4]},discrete = value,)
exp = fit.power_law.alpha
axd.annotate("Pooled alpha= {}".format(round(exp,2)),[0.8,0.5], xycoords = 'axes fraction', fontsize = 11)
pwl.plot_pdf(data,color = 'tab:green', marker = '.', ax = axd)
fit.power_law.plot_pdf(data,color = 'tab:green' ,linestyle = '--', ax = axd)

        
plt.tight_layout()